In [4]:
from __future__ import print_function

import copy
import os.path as osp
import os
import sys
sys.path.insert(0, os.path.abspath('/home/Various_CAM/grad-cam-pytorch'))

import click
import cv2
import matplotlib.cm as cm
import numpy as np
import torch
import torch.hub
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import models, transforms
from collections import OrderedDict


from grad_cam import (
    BackPropagation,
    Deconvnet,
    GradCAM,
    GuidedBackPropagation,
    occlusion_sensitivity,
)

# if a model includes LSTM, such as in image captioning,
# torch.backends.cudnn.enabled = False

In [2]:
import torch, os
from torchvision import transforms
from torch.autograd import Variable
from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import numpy as np
import cv2, torch
import numpy as np
import pandas as pd
import io
import requests
from PIL import Image
from collections import OrderedDict
from skimage import img_as_float,io

In [5]:
def get_device(cuda):
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device


def get_classtable():
    classes = []
    with open("samples/cancer.txt") as lines:
        for line in lines:
            line = line.strip().split(" ", 1)[1]
            line = line.split(", ", 1)[0].replace(" ", "_")
            classes.append(line)
    return classes


def preprocess(image_path, img_size):
#     raw_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     1D
    raw_image = np.empty(shape=(1, img_size * img_size))
    raw_image[0, :] = (img_as_float(io.imread(image_path)) - 0.5)/0.5
    
#     2D
#     raw_image = np.empty(shape=(1, 96, 96))
#     raw_image[0, :, :] = (img_as_float(io.imread(image_path)) - 0.5)/0.5
    
    
#     raw_image = cv2.resize(raw_image, (224,) * 2)
    image = transforms.Compose(
        [
            transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )(raw_image[..., ::-1].copy())
    image = image.type(torch.cuda.DoubleTensor)
    return image, raw_image


def save_gradient(filename, gradient):
    gradient = gradient.cpu().numpy().transpose(0, 1)
    gradient -= gradient.min()
    if gradient.max() != 0:
        gradient /= gradient.max()
        gradient *= 255.0
    else:
        gradient = 0
    cv2.imwrite(filename, np.uint8(gradient))
    grads = np.uint8(gradient)
    return grads


def save_gradcam(filename, gcam, raw_image, paper_cmap=False):
    gcam = gcam.cpu().numpy()
    gcam = cv2.applyColorMap(np.uint8(gcam * 255.0), cv2.COLORMAP_JET)
    # gcam = gcam.astype(np.float) + raw_image.astype(np.float)
    if np.nanmax(gcam) != 0:
        gcam = (gcam / np.nanmax(gcam)) * 255.0
    else:
        gcam = 0
    cv2.imwrite(filename, np.uint8(gcam))
#     cmap = cm.jet_r(gcam)[..., :3] * 255.0
#     if paper_cmap:
#         alpha = gcam[..., None]
#         gcam = alpha * cmap + (1 - alpha) * raw_image
#     else:
#         gcam = (cmap.astype(np.float) + raw_image.astype(np.float)) / 2
#     cv2.imwrite(filename, np.uint8(gcam))


def save_sensitivity(filename, maps):
    maps = maps.cpu().numpy()
    scale = max(maps[maps > 0].max(), -maps[maps <= 0].min())
    maps = maps / scale * 0.5
    maps += 0.5
    maps = cm.bwr_r(maps)[..., :3]
    maps = np.uint8(maps * 255.0)
    maps = cv2.resize(maps, (224, 224), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(filename, maps)

In [ ]:
#1 dimension
class Net(nn.Module):

        def __init__(self, num_of_classes):
            super(Net, self).__init__()
            # input image channel, output channels, kernel size square convolution
            # kernel
            # input size = 28//96 output size = 28//96
            self.conv1 = nn.Conv1d(1, 64, kernel_size=kernel_set, padding=1)
            self.bn1 = nn.BatchNorm1d(64)
            self.relu1 = nn.ReLU(inplace = False)
            # input size = 14//48 output size = 6//48
            self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm1d(128)
            self.relu2 = nn.ReLU(inplace = False)
            # input size = 6//24, output size = 3//24
            self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
            self.bn3 = nn.BatchNorm1d(256)
            self.relu3 = nn.ReLU(inplace = False)
            self.drop = nn.Dropout(p=0.25, inplace=False)
            self.vp = nn.MaxPool1d(kernel_size=4, padding=0, stride=4)
            # an affine operation: y = Wx + b
            self.fc1 = nn.Linear(256*final_feature, 1024)
            self.fc2 = nn.Linear(1024, 512)
            self.fc3 = nn.Linear(512, 256)
            self.fc4 = nn.Linear(256, num_of_classes)


        def forward(self, x):
            in_size = x.size(0)
            x = self.relu1(self.bn1(self.vp(self.conv1(x))))
            x = self.relu2(self.bn2(self.vp(self.conv2(x))))
            x = self.relu3(self.bn3(self.vp(self.conv3(x))))
            x = self.drop(x)
            x = x.view(in_size, -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.fc3(x)
            x = self.fc4(x)
            return x

In [7]:
# @main.command()
# @click.option("-i", "--image-paths", type=str, multiple=True, required=True)
# @click.option("-a", "--arch", type=click.Choice(model_names), required=True)
# @click.option("-t", "--target-layer", type=str, required=True)
# @click.option("-k", "--topk", type=int, default=2)
# @click.option("-o", "--output-dir", type=str, default="./results")
# @click.option("--cuda/--cpu", default=True)

def demo1(image_paths, target_layer, topk, output_dir, cuda, trained_loc, img_size = 98):
    """
    Visualize model responses given multiple images
    """
    arch = 'resnet152'
    device = get_device(cuda)

    # Synset words
    classes = get_classtable()

    # Model from torchvision
#     model = models.resnet152(pretrained=True)
    model = Net(num_of_classes=6)
    model = model.double()
    
    state_dict = torch.load(trained_loc)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    model.load_state_dict(new_state_dict)
    print('Weight loaded')
#     model.cuda()
    model.to(device)
    model.eval()

    # Images
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate(image_paths):
#         print("\t#{}: {}".format(i,image_path))
        image, raw_image = preprocess(image_path, img_size)
        images.append(image)
        raw_images.append(raw_image)
    print(len(images))
    img_count = len(images)
    images = torch.stack(images).to(device)
    print('before: ',images.size())
    images = torch.reshape(images,(img_count,1,img_size*img_size))
    print('after: ',images.size())
    
    image_files = os.listdir(folder)
    for fichier in image_files[:]: # filelist[:] makes a copy of filelist.
        if not(fichier.endswith(".png")):
            image_files.remove(fichier)

    """
    Common usage:
    1. Wrap your model with visualization classes defined in grad_cam.py
    2. Run forward() with images
    3. Run backward() with a list of specific classes
    4. Run generate() to export results
    """

    # =========================================================================
    print("Vanilla Backpropagation:")

    bp = BackPropagation(model=model)
    probs, ids = bp.forward(images)

#     for i in range(topk):
#         # In this example, we specify the high confidence classes
#         bp.backward(ids=ids[:, [i]])
#         gradients = bp.generate()

        # Save results as image files
#         for j in range(len(images)):
#             print("\t#{}: {} ({:.5f})".format(image_files[j], classes[ids[j, i]], probs[j, i]))

#             save_gradient(
#                 filename=osp.join(
#                     output_dir,
#                     "{}-{}-vanilla-{}.png".format(classes[ids[j, i]], image_files[j], arch ),
#                 ),
#                 gradient=gradients[j],
#             )

    # Remove all the hook function in the "model"
    bp.remove_hook()

#     # =========================================================================
#     print("Deconvolution:")

#     deconv = Deconvnet(model=model)
#     _ = deconv.forward(images)

#     for i in range(topk):
#         deconv.backward(ids=ids[:, [i]])
#         gradients = deconv.generate()

#         for j in range(len(images)):
# #             print("\t#{}: {} ({:.5f})".format(image_files[j], classes[ids[j, i]], probs[j, i]))

#             save_gradient(
#                 filename=osp.join(
#                     output_dir,
#                     "{}-{}-deconvnet-{}.png".format(classes[ids[j, i]], image_files[j], arch, ),
#                 ),
#                 gradient=gradients[j],
#             )

#     deconv.remove_hook()

    # =========================================================================
    print("Grad-CAM/Guided Backpropagation/Guided Grad-CAM:")

    gcam = GradCAM(model=model)
    _ = gcam.forward(images)

    gbp = GuidedBackPropagation(model=model)
    _ = gbp.forward(images)
    

    for i in range(topk):
        # Guided Backpropagation
        gbp.backward(ids=ids[:, [i]])
        gradients = gbp.generate()

        # Grad-CAM
        gcam.backward(ids=ids[:, [i]])
        regions = gcam.generate(target_layer=target_layer)
        
        for j in range(len(images)):
#             print("\t#{}: {} ({:.5f})".format(image_files[j], classes[ids[j, i]], probs[j, i]))
            
            # Guided Backpropagation
            save_gradient(
                filename=osp.join(
                    output_dir,
                    "guided_{}_{}_{:.3f}.png".format(classes[ids[j, i]], image_files[j], probs[j,i]),
                ),
                gradient=gradients[j],
            )
#             print(classes[ids[j, i]],image_files[j])
            # Guided Grad-CAM
            save_gradient(
                filename=osp.join(
                    output_dir,
                    "guided-gradcam_{}_{}_{:.3f}_{}.png".format(
                        classes[ids[j, i]], image_files[j], probs[j,i], target_layer 
                    ),
                ),
                gradient=torch.mul(regions, gradients)[j],
            )
            
#             Grad-CAM
            save_gradcam(
                filename=osp.join(
                    output_dir,
                    "gradcam_{}_{}_{:.3f}_{}.png".format(
                        classes[ids[j, i]], image_files[j], probs[j,i], target_layer 
                    ),
                ),
                gcam=regions[j, 0],
                raw_image=raw_images[j],
            )
    
def demoguided(image_paths, target_layer, topk, output_dir, cuda, trained_loc):
    """
    Visualize model responses given multiple images
    """
    arch = 'resnet152'
    device = get_device(cuda)

    # Synset words
    classes = get_classtable()

    # Model from torchvision
#     model = models.resnet152(pretrained=True)
    model = Net(num_of_classes=6)
    model = model.double()
    
    state_dict = torch.load(trained_loc)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    model.load_state_dict(new_state_dict)
    print('Weight loaded')
#     model.cuda()
    model.to(device)
    model.eval()

    # Images
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate(image_paths):
#         print("\t#{}: {}".format(i,image_path))
        image, raw_image = preprocess(image_path)
        images.append(image)
        raw_images.append(raw_image)
    print(len(images))
    images = torch.stack(images).to(device)
    
    image_files = os.listdir(folder)
    for fichier in image_files[:]: # filelist[:] makes a copy of filelist.
        if not(fichier.endswith(".png")):
            image_files.remove(fichier)

    """
    Common usage:
    1. Wrap your model with visualization classes defined in grad_cam.py
    2. Run forward() with images
    3. Run backward() with a list of specific classes
    4. Run generate() to export results
    """

    # =========================================================================
    print("Vanilla Backpropagation:")

    bp = BackPropagation(model=model)
    probs, ids = bp.forward(images)

    for i in range(topk):
        # In this example, we specify the high confidence classes
        bp.backward(ids=ids[:, [i]])
        gradients = bp.generate()

        # Save results as image files
#         for j in range(len(images)):
# #             print("\t#{}: {} ({:.5f})".format(image_files[j], classes[ids[j, i]], probs[j, i]))

#             save_gradient(
#                 filename=osp.join(
#                     output_dir,
#                     "{}-{}-vanilla-{}.png".format(classes[ids[j, i]], image_files[j], arch ),
#                 ),
#                 gradient=gradients[j],
#             )

    # Remove all the hook function in the "model"
    bp.remove_hook()

# #     # =========================================================================
    print("Deconvolution:")

    deconv = Deconvnet(model=model)
    _ = deconv.forward(images)

    for i in range(topk):
        deconv.backward(ids=ids[:, [i]])
        gradients = deconv.generate()

#         for j in range(len(images)):
# #             print("\t#{}: {} ({:.5f})".format(image_files[j], classes[ids[j, i]], probs[j, i]))

#             save_gradient(
#                 filename=osp.join(
#                     output_dir,
#                     "{}-{}-deconvnet-{}.png".format(classes[ids[j, i]], image_files[j], arch, ),
#                 ),
#                 gradient=gradients[j],
#             )

    deconv.remove_hook()

#     # =========================================================================
    print("Grad-CAM/Guided Backpropagation/Guided Grad-CAM:")

    gcam = GradCAM(model=model)
    _ = gcam.forward(images)

    gbp = GuidedBackPropagation(model=model)
    _ = gbp.forward(images)
    

    for i in range(topk):
        # Guided Backpropagation
        gbp.backward(ids=ids[:, [i]])
        gradients = gbp.generate()

        # Grad-CAM
        gcam.backward(ids=ids[:, [i]])
        regions = gcam.generate(target_layer=target_layer)
        
        for j in range(len(images)):
#             print("\t#{}: {} ({:.5f})".format(image_files[j], classes[ids[j, i]], probs[j, i]))
            
#             # Guided Backpropagation
#             save_gradient(
#                 filename=osp.join(
#                     output_dir,
#                     "{}_{}_guided_{:.3f}.png".format(classes[ids[j, i]], image_files[j], probs[j,i]),
#                 ),
#                 gradient=gradients[j],
#             )
            
            # Guided Grad-CAM
            save_gradient(
                filename=osp.join(
                    output_dir,
                    "{}_{}_guided_gradcam_{:.3f}_{}.png".format(
                        classes[ids[j, i]], image_files[j], probs[j,i], target_layer 
                    ),
                ),
                gradient=torch.mul(regions, gradients)[j],
            )
            
#             Grad-CAM
#             save_gradcam(
#                 filename=osp.join(
#                     output_dir,
#                     "{}_{}_gradcam_{:.3f}_{}.png".format(
#                         classes[ids[j, i]], image_files[j], probs[j,i], target_layer 
#                     ),
#                 ),
#                 gcam=regions[j, 0],
#                 raw_image=raw_images[j],
#             )
# @main.command()
# @click.option("-i", "--image-paths", type=str, multiple=True, required=True)
# @click.option("-o", "--output-dir", type=str, default="./results")
# @click.option("--cuda/--cpu", default=True)
def demo2(image_paths, output_dir, cuda, trained_loc):
    """
    Generate Grad-CAM at different layers of ResNet-152
    """

    device = get_device(cuda)

    # Synset words
    classes = get_classtable()

    # Model
    model = Net(num_of_classes=6)
    model = model.double()
    #load_weight
    state_dict = torch.load(trained_loc)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    model.load_state_dict(new_state_dict)
    print('Weight loaded')
    
#     model = models.resnet152(pretrained=True)
    model.to(device)
    model.eval()

    # The four residual layers
    target_layers = ["relu3","conv3", "fc1"]
    target_class = 1

    # Images
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate(image_paths):
        print("\t#{}: {}".format(i, image_path))
        image, raw_image = preprocess(image_path)
        images.append(image)
        raw_images.append(raw_image)
    images = torch.stack(images).to(device)

    gcam = GradCAM(model=model)
    probs, ids = gcam.forward(images)
    ids_ = torch.LongTensor([[target_class]] * len(images)).to(device)
    gcam.backward(ids=ids_)

    for target_layer in target_layers:
        print("Generating Grad-CAM @{}".format(target_layer))

        # Grad-CAM
        regions = gcam.generate(target_layer=target_layer)

        for j in range(len(images)):
            print(
                "\t#{}: {} ({:.5f})".format(
                    j, classes[target_class], float(probs[ids == target_class])
                )
            )

            save_gradcam(
                filename=osp.join(
                    output_dir,
                    "{}-{}-gradcam-{}-{}.png".format(
                        j, "resnet152", target_layer, classes[target_class]
                    ),
                ),
                gcam=regions[j, 0],
                raw_image=raw_images[j],
            )